# Trade Execution

Loads the trade plan from `01_analysis_pipeline.ipynb`, allows custom edits,
and executes via IB Gateway with trailing stops on all BUY fills.

| Step | Description |
|------|-------------|
| 1 | Load trade plan |
| 2 | Edit instructions (optional) |
| 3 | Interpret instructions |
| 4 | Execute (CONFIRM=True to go live) |

---
## Step 1: Load Trade Plan

In [ ]:
import sys
from pathlib import Path

import nest_asyncio
import pandas as pd
nest_asyncio.apply()

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))
sys.path.insert(0, str(PROJECT_ROOT / "notebooks"))

LIVE_DIR = Path.home() / "trading" / "live_portfolio"
TRADE_PLAN_FILE = LIVE_DIR / "trade_plan.csv"

IB_HOST = "127.0.0.1"
IB_PORT = 4001
IB_CLIENT_ID = 15
TRAILING_STOP_PCT = 10

from scripts.s7_execute import load_trade_plan

trades = load_trade_plan(TRADE_PLAN_FILE)
print(f"Loaded {len(trades)} trades from {TRADE_PLAN_FILE}")
pd.DataFrame(trades)

---
## Step 2: Edit Instructions (Optional)

Edit `custom_instructions` below. Examples:
- `"BND": "SKIP"` — skip this trade
- `"DIM": "reduce to 30 shares"` — override share count
- `"EWU": "limit at $45"` — use limit order

Anything not listed stays as `APPROVE`.

In [ ]:
custom_instructions = {
    # "BND": "SKIP",
    # "DIM": "reduce to 30 shares",
}

from scripts.s7_execute import apply_custom_instructions

trades = apply_custom_instructions(trades, custom_instructions)
pd.DataFrame(trades)

---
## Step 3: Interpret Instructions

Uses Claude API if `ANTHROPIC_API_KEY` is set, otherwise keyword fallback.

In [ ]:
from scripts.s7_execute import interpret_trades

final_trades = interpret_trades(trades)
print(f"{len(final_trades)} trades ready for execution")
pd.DataFrame(final_trades)

---
## Step 4: Execute Trades

**Set `CONFIRM = True` to place REAL orders on IB.**

Trailing stops (10% TRAIL, GTC) are placed automatically on every BUY fill.

In [ ]:
# ============================================
CONFIRM = False  # Set True to execute
# ============================================

from scripts.s7_execute import execute_trades

exec_results = execute_trades(
    final_trades, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
    trailing_stop_pct=TRAILING_STOP_PCT,
    confirm=CONFIRM,
)

if exec_results:
    pd.DataFrame(exec_results)